In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup, Comment
import pandas as pd

In [ ]:
def append_college_stats(rows):
    player_stats = []
    for i in range(len(rows)):
        player = [td.getText() for td in rows[i].findAll('td')]
        if player == []:
            continue
        td_list = rows[i].findAll('td')
    for i in range(len(rows)):
        player = [td.getText() for td in rows[i].findAll('td')]
        if player == []:
            continue
        td_list = rows[i].findAll('td')
        for td in td_list:  
            if td['data-stat'] == "player":
                button = td.findAll('a')
                if button != []:
                    link = button[0]['href']
                    player_url = "https://www.basketball-reference.com{}".format(link)
                    player_html = urlopen(player_url)
                    player_soup = BeautifulSoup(player_html)
                    college_stats = player_soup.findAll(id="all_all_college_stats")
                    player_height = player_soup.findAll("span",{'itemprop': 'height'})[0].text
                    player_weight = player_soup.findAll("span",{'itemprop':'weight'})[0].text
                    birth_date = player_soup.findAll("div",{"itemtype":"https://schema.org/Person"})[0].findAll("span",{"id":"necro-birth"})[0].text.replace('\n','').strip()
                    birth_place = player_soup.findAll("div",{"itemtype":"https://schema.org/Person"})[0].find("span",{"itemprop":"birthPlace"}).a.text                    
                    if college_stats != []:
                        for comments in college_stats[0].findAll(text=lambda text:isinstance(text, Comment)):
                            x = comments.extract()
                            comment_soup = BeautifulSoup(x , 'lxml')
                            tfoot = comment_soup.findAll('tfoot')
                            td = tfoot[0].findAll('td')
                            for i in range(2, len(td)):
                                player.append(td[i].getText())
                            player.append(player_height)
                            player.append(player_weight)
                            player.append(birth_date)
                            player.append(birth_place)
        player_stats.append(player)
        print('PROCESSING PLAYER {}'.format(player_url))
    return player_stats
        
    

In [ ]:
# Defining range of draft classes
start_year = 2007
end_year = 2019
college_headers = ['college_G',
                   'college_MP',
                   'college_FG',
                    'college_FGA',
                    'college_3P',
                    'college_3PA',
                    'college_FT',
                    'college_FTA',
                    'college_ORB',
                    'college_TRB',
                    'college_AST',
                    'college_STL',
                    'college_BLK',
                    'college_TOV',
                    'college_PF',
                    'college_PTS',
                    'college_FG%',
                    'college_3P%',
                    'college_FT%',
                    'college_MP',
                    'college_PTS%',
                    'college_TRB',
                    'college_AST',
                   'height',
                   'weight',
                   'date_of_birth',
                   'place_of_birth'
                  ]

# Initialize .csv writer
writer = pd.ExcelWriter('nba_draft.xlsx', engine='xlsxwriter')

# Push data to .csv file (1 sheet per draft class)
for i in range(start_year, end_year + 1):
    url = "https://www.basketball-reference.com/draft/NBA_{}.html".format(i)
    html = urlopen(url)
    soup = BeautifulSoup(html)
    rows = soup.findAll('tr', limit=2)
    headers = [th.getText() for th in rows[1].findAll('th')]
    headers = headers[1:]
    headers = headers + college_headers    
    rows = soup.findAll('tr')[2:]
    player_stats = append_college_stats(rows)
    stats = pd.DataFrame(player_stats, columns = headers)
    stats = stats.dropna()
    data = stats.head(60)
    stats.to_excel(writer, sheet_name='draft_data_{}'.format(i), index = False)
    
# Save .csv data
writer.save()